In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import requests
import re
from bs4 import BeautifulSoup

In [16]:
class pipeline():
    def __init__(self, medium_content_url, module_url):
        assert type(medium_content_url) == str
        assert type(module_url) == str
        self.medium_content_url = medium_content_url
        self.module_url = module_url
        self.__prepare_text_from_html(self.medium_content_url)
        self.__loading_module(self.module_url)
        self.__text_embedded = self.run_embedding(self.embed, self.text)
    
    # function for downloading content from the web
    def __content_downloader(self, url):
        request = requests.get(url)
        soup = BeautifulSoup(request.content, "lxml") 
        self._html = str(soup.find("div", {"class": "postArticle-content js-postField js-notesSource js-trackedPost"}))
    
    # cleaning the content and return sentences it's not compelete and need to be modified
    def __cleaning_text(self, html_part):
        text = re.sub('<[A-Za-z\/][^>]*>', ' ', str(html_part))
        text = re.split(r'\s{2,}', text)[1:-1]
        text = [sentence.replace('\xa0', ' ') for sentence in text]
        text = [sentence.replace('\u200a—\u200a', ' ') for sentence in text]
        return text
    
    # preparing a text for execute caculations on it
    def __prepare_text_from_html(self, url):
        self.__content_downloader(url)
        self.text = self.__cleaning_text(self._html)
    
    # function for loading diffrenet module
    def __loading_module(self, module_url):
        # Import the Universal Sentence Encoder's TF Hub module
        self.embed_object = hub.Module(module_url)
        
    # function for runinng embedding module on text
    def __run_embedding(embed_object, text):
        # Reduce logging output.
        tf.logging.set_verbosity(tf.logging.ERROR)

        with tf.Session() as session:
            session.run([tf.global_variables_initializer(), tf.tables_initializer()])
            message_embeddings = session.run(embed_object(text))

        return message_embeddings
    
    # function for asking question from text
    def ask_question_from_text(self, question):
        self.question = list(question)
        self.question_embedded = self.run_embedding(self.embed, self.question)
    
    # function for calculating similarity between question and text
    def calculating_similarity_tensor(self):
        question_tensor = tf.Variable(tf.convert_to_tensor(self.question_embedded))
        text_tensor = tf.Variable(tf.convert_to_tensor(self.text_embedded))
        multiply_tensor = tf.matmul(question_tensor, text_tensor, transpose_b = True)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            answer = sess.run(multiply_tensor)
        self.similarity_tensor
                                  
    # function for find sentence in text that answer question that has been asked
    def find_the_most_similar_sentence(self, similarity_tensor):
        print('similarity score for the most similar sentence is {}'.format(np.max(similarity_tensor)))
        return np.hstack([question, text[np.argmax(similarity_tensor)]]).reshape(-1, 1)